# <font color=green> **AULA PRÁTICA 8**
-----



#**Retail Demand Forecasting**


A referência do tema foi adquirida na página:

https://towardsdatascience.com/machine-learning-for-store-demand-forecasting-and-inventory-optimization-part-1-xgboost-vs-9952d8303b48

<br>

O artigo do site propõe a resolução do problema de **"Otimização de Planejamento de Demanda"** com a previsão da demanda de cada produto de cada uma das lojas.

Nesta aula, iremos reduzir e utilizar os dados fornecidos para mostrar como podemos avaliar séries temporais e propor modelos de previsão de demanda.

**OBS:** O objetivo não é resolver o problema do desafio, mas fornecer uma introdução em como podemos lidar com séries temporais e previsões.

## 1 - Descrição dos Dados

**Onde acessar?**

* https://www.kaggle.com/c/demand-forecasting-kernels-only/data

<br> 

**Qual o objetivo da competição?**

* Prever 3 meses da venda de um determinado item em diferentes localizações da loja

<br>

**Quais dados são disponibilizados?**

* Training data test.csv
* Test data (Note: the Public/Private split is time based) 
* Sample_submission.csv - a sample submission file in the correct format

<br>

**Quais são as variáveis?**

* **date** - Date of the sale data. There are no holiday effects or store closures. 
* **store** - Store ID item 
* **Item ID** 
* **sales** - Number of items sold at a particular store on a particular date.



## 2 - Importando os dados

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt           
import seaborn as sns                    
import warnings 
warnings.filterwarnings("ignore") 

In [2]:
# lendo arquivos de treino  e teste
df_treino = pd.read_csv('/content/drive/Shareddrives/Disciplina Modelagem/Aulas/Aula 8/train.csv')
df_teste = pd.read_csv('/content/drive/Shareddrives/Disciplina Modelagem/Aulas/Aula 8/test.csv')

In [3]:
df_treino.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [4]:
df_teste.head()

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1


Notar que o conjunto de teste não tem "sales". Ou seja, esse dataframe é apenas uma amostras das datas do desafio que deverão ser previstas. Para o que queremos analisar, não será um conjunto útil.

## 3- Análise Exploratória dos dados

In [5]:
# tamanho da série de treino
df_treino.shape

(913000, 4)

In [6]:
# elementos na coluna store? são então 10 lojas, com 91300 registros cada
df_treino['store'].value_counts()

10    91300
9     91300
8     91300
7     91300
6     91300
5     91300
4     91300
3     91300
2     91300
1     91300
Name: store, dtype: int64

In [7]:
## quantos itens diferentes?
print(f"Número de itens diferentes: {len(df_treino['item'].unique())}")
df_treino['item'].unique()

Número de itens diferentes: 50


array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])

In [8]:
# transformando as datas para time
df_treino['date'] = pd.to_datetime(df_treino['date'])   # sempre precisa passar data para esse formato
df_treino.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [9]:
# RESUMO ESTATÍSTICO
df_treino.describe()

,store,item,sales
count,913000.000000,913000.000000,913000.000000
mean,5.500000,25.500000,52.250287
std,2.872283,14.430878,28.801144
min,1.000000,1.000000,0.000000
25%,3.000000,13.000000,30.000000
50%,5.500000,25.500000,47.000000
75%,8.000000,38.000000,70.000000
max,10.000000,50.000000,231.000000


In [11]:
df_treino.head()

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [14]:
# qual seria a venda por ano? e por mês? por dia?
df_treino['date'] = pd.to_datetime(df_treino['date'])
df_treino['year'] = df_treino.date.dt.year              # da variável date, pega o ano
df_treino['month'] = df_treino.date.dt.month            # da variável date, pega o mês
df_treino['day'] = df_treino.date.dt.day                # da variável date, pega o dia
df_treino['dayofyear'] = df_treino.date.dt.dayofyear      # da variável date, pega o dia do ano
df_treino['dayofweek'] = df_treino.date.dt.dayofweek    # da variável date, pega o dia da semana  - Monday, which is denoted by 0 and ends on Sunday which is denoted by 6
df_treino['weekofyear'] = df_treino.date.dt.weekofyear   # da variável date, pega o dia da semana do ano

In [15]:
# o que significa primeira semana do ano?
df_treino.loc[df_treino['weekofyear'] == 1,:].loc[0:400,:]

,date,store,item,sales,year,month,day,dayofyear,dayofweek,weekofyear
0,2013-01-01,1,1,13,2013,1,1,1,1,1
1,2013-01-02,1,1,11,2013,1,2,2,2,1
2,2013-01-03,1,1,14,2013,1,3,3,3,1
3,2013-01-04,1,1,13,2013,1,4,4,4,1
4,2013-01-05,1,1,10,2013,1,5,5,5,1
5,2013-01-06,1,1,12,2013,1,6,6,6,1
363,2013-12-30,1,1,5,2013,12,30,364,0,1
364,2013-12-31,1,1,15,2013,12,31,365,1,1
365,2014-01-01,1,1,9,2014,1,1,1,2,1
366,2014-01-02,1,1,14,2014,1,2,2,3,1


In [16]:
# o que a gente faz para treino, a gente tem que fazer para teste:
def time_operations(df):
    # qual seria a venda por ano? e por mês? por dia?
  df['date'] = pd.to_datetime(df['date'])
  df['year'] = df.date.dt.year              # da variável date, pega o ano
  df['month'] = df.date.dt.month            # da variável date, pega o mês
  df['day'] = df.date.dt.day                # da variável date, pega o dia
  df['dayofyear'] = df.date.dt.dayofyear      # da variável date, pega o dia do ano
  df['dayofweek'] = df.date.dt.dayofweek    # da variável date, pega o dia da semana  - Monday, which is denoted by 0 and ends on Sunday which is denoted by 6
  df['weekofyear'] = df.date.dt.weekofyear   # da variável date, pega o dia da semana do ano
  return(df)

df_treino = time_operations(df_treino)

In [17]:
df_treino

,date,store,item,sales,year,month,day,dayofyear,dayofweek,weekofyear
0,2013-01-01,1,1,13,2013,1,1,1,1,1
1,2013-01-02,1,1,11,2013,1,2,2,2,1
2,2013-01-03,1,1,14,2013,1,3,3,3,1
3,2013-01-04,1,1,13,2013,1,4,4,4,1
4,2013-01-05,1,1,10,2013,1,5,5,5,1
...,...,...,...,...,...,...,...,...,...,...
912995,2017-12-27,10,50,63,2017,12,27,361,2,52
912996,2017-12-28,10,50,59,2017,12,28,362,3,52
912997,2017-12-29,10,50,74,2017,12,29,363,4,52
912998,2017-12-30,10,50,62,2017,12,30,364,5,52


### Olhando apenas para o item 1

A fim de simplificar nosso problema, vamos verificar apenas o item 1.

In [18]:
df_item1 = df_treino.loc[df_treino['item']==1,:]  # filtrando pelo item 1
df_item1.head()

,date,store,item,sales,year,month,day,dayofyear,dayofweek,weekofyear
0,2013-01-01,1,1,13,2013,1,1,1,1,1
1,2013-01-02,1,1,11,2013,1,2,2,2,1
2,2013-01-03,1,1,14,2013,1,3,3,3,1
3,2013-01-04,1,1,13,2013,1,4,4,4,1
4,2013-01-05,1,1,10,2013,1,5,5,5,1


In [19]:
df_item1.shape

(18260, 10)

Gráficos interativos

In [20]:
# gráfico interativo
import plotly.graph_objects as go
import plotly.express as px

fig2 = px.line(df_item1, x='date', y='sales', color="store",color_discrete_sequence=px.colors.qualitative.G10)
fig2.update_layout(height=400, width=1100,template="plotly_white",title_text='Variável ao longo do tempo')
fig2.update_traces(marker=dict(size=8,
                              line=dict(width=0.08,
                                        color='Grey')),
                  selector=dict(mode='markers'))


fig2.show(renderer="colab")

A partir do gráfico acima, o que podemos perceber?

Um outro ponto interessante é que muitas vezes precisamos mudar a frequência dos nossos dados. Mas o que significa mudar a frequência dos dados da série temporal? 

Ou seja, mudar de mês para ano, ou minuto para hora. Se faz sentido prever seus dados a cada mês, e eles estão com intervalo de dia, precisamos passar de dias para mês, escolhendo alguma medida apropriada.


Podemos usar a função **resample** do pandas, exclusiva para séries temporais.

In [21]:
# Para loja 1:
store1 = 1
df_item1.loc[df_item1['store'] == store1,:].head()

,date,store,item,sales,year,month,day,dayofyear,dayofweek,weekofyear
0,2013-01-01,1,1,13,2013,1,1,1,1,1
1,2013-01-02,1,1,11,2013,1,2,2,2,1
2,2013-01-03,1,1,14,2013,1,3,3,3,1
3,2013-01-04,1,1,13,2013,1,4,4,4,1
4,2013-01-05,1,1,10,2013,1,5,5,5,1


In [ ]:
# Exemplo de como podemos transformar para outra frequência com resample
df_item1.loc[df_item1['store'] == store1,:].resample('M', on= 'date',convention ='start')['sales'].mean()

In [25]:
df_novo_treino = pd.DataFrame(columns = ['date','store','sales'])   # dataframe vazio

store = df_item1['store'].unique()   # todas as lojas diferentes

for store1 in store:

  # treino
  df_temp = pd.DataFrame(df_item1.loc[df_item1['store'] == store1,:].resample('W', on= 'date',convention ='start')['sales'].sum())   # resample para Mês
  df_temp['date'] = df_temp.index   # coluna data
  df_temp['store'] = len(df_temp)*[store1]   # coluna de loja
  df_temp = df_temp[['date','store','sales']]    # colocando as colunas em ordem
  df_novo_treino = pd.concat([df_novo_treino, df_temp],ignore_index= 'True')   # concatenando dataframes


 ## Criando um gráfico no plotly           
fig2 = px.line(df_novo_treino, x='date', y='sales', color="store",color_discrete_sequence=px.colors.qualitative.G10)
fig2.update_layout(height=400, width=1100,template="plotly_white",title_text='Variável ao longo do tempo')
fig2.update_traces(marker=dict(size=8,
                              line=dict(width=0.08,
                                        color='Grey')),
                  selector=dict(mode='markers'))

fig2.show(renderer="colab")

O que podemos observar pelos gráficos?

- há sazonalidade: os períodos de julho tem maiores vendas e os períodos de janeiro menores vendas.

- Todas lojas tem a mesma tendência durante o período (mesma crista, vale, e possivelmente mesmo crescimento ao longo do tempo).

Existe diferença de venda do produto 1 por dia da semana?

In [26]:
fig2 = px.box(df_item1, x='dayofweek', y='sales', color="store",color_discrete_sequence=px.colors.qualitative.G10)
fig2.update_layout(height=400, width=1100,template="plotly_white",title_text='Variável ao longo do tempo')
fig2.update_traces(marker=dict(size=8,
                              line=dict(width=0.08,
                                        color='Grey')),
                  selector=dict(mode='markers'))


fig2.show(renderer="colab")

O que se pode observar pelo gráfico acima?

E em relação as vendas ao longo do tempo?

In [27]:

fig2 = px.bar(df_item1, x='store', y='sales', facet_col="year", barmode="group")
fig2.update_layout(height=400, width=1100,template="plotly_white",title_text='Variável ao longo do tempo')
fig2.update_traces(marker=dict(size=8,
                              line=dict(width=0.08,
                                        color='Grey')),
                  selector=dict(mode='markers'))


fig2.show(renderer="colab")

In [28]:
df_item1.loc[df_item1['store']==1,:].groupby(['month']).mean()

,store,item,sales,year,day,dayofyear,dayofweek,weekofyear
month,,,,,,,,
1,1.0,1.0,13.709677,2015.000000,16.000000,16.000000,3.025806,4.129032
2,1.0,1.0,14.631206,2015.007092,14.602837,45.602837,2.978723,7.000000
3,1.0,1.0,17.600000,2015.000000,16.000000,75.200000,3.019355,11.225806
4,1.0,1.0,20.786667,2015.000000,15.500000,105.700000,2.993333,15.586667
5,1.0,1.0,22.245161,2015.000000,16.000000,136.200000,3.006452,19.941935
6,1.0,1.0,24.833333,2015.000000,15.500000,166.700000,3.000000,24.300000
7,1.0,1.0,25.967742,2015.000000,16.000000,197.200000,2.993548,28.658065
8,1.0,1.0,22.529032,2015.000000,16.000000,228.200000,3.012903,33.083871
9,1.0,1.0,21.386667,2015.000000,15.500000,258.700000,2.973333,37.446667


Tem diferença de venda por mês? Como que podemos ver isso para loja 1?

In [29]:
# Para a loja 1:
loja = 1
df_graf = df_item1.loc[df_item1['store']==1,:].groupby(['month']).mean()  # média de vendas mensais da loja 1 -item 1 durante os anos

fig2 = px.bar(df_graf, x=df_graf.index, y='sales', color_discrete_sequence=px.colors.qualitative.G10)
fig2.update_layout(height=400, width=1100,template="plotly_white",title_text='Variável ao longo do tempo')
fig2.update_traces(marker=dict(size=8,
                              line=dict(width=0.08,
                                        color='Grey')),
                  selector=dict(mode='markers'))


fig2.show(renderer="colab")

Podemos perceber que junho e julho são os meses com maiores vendas

Pode ser um período de verão, período de promoção, diferentes comportamentos. O que é importante é atentar que existe uma sazonalidade.

## 4- Análise de séries temporais

Para o estudo, iremos seguir com a série temporal do item 1 e loja 1. 

In [30]:
df_item1.head(3)

,date,store,item,sales,year,month,day,dayofyear,dayofweek,weekofyear
0,2013-01-01,1,1,13,2013,1,1,1,1,1
1,2013-01-02,1,1,11,2013,1,2,2,2,1
2,2013-01-03,1,1,14,2013,1,3,3,3,1


In [33]:
df_item1.shape

(18260, 10)

In [31]:
# Criando um dataframe store= 1 e item = 1
df = df_item1.loc[df_item1['store']==1,:]
df.shape

(1826, 10)

In [35]:
# Plotando novamente a série temporal do item 1 e agora só loja 1
# gráfico interativo
import plotly.graph_objects as go
import plotly.express as px

fig2 = px.line(df, x='date', y='sales', color_discrete_sequence=px.colors.qualitative.G10)
fig2.update_layout(height=400, width=1100,template="plotly_white",title_text='Variável ao longo do tempo')
fig2.update_traces(marker=dict(size=8,
                              line=dict(width=0.08,
                                        color='Grey')),
                  selector=dict(mode='markers'))


fig2.show(renderer="colab")

**SÉRIES TEMPORAIS:**

<br>

- O que são?

  - São séries que contém **padrões não aleatórios**. Ou seja, sistema causal com uma determinada constância relacionada ao tempo.

<br>

- O que significa fazer uma análise de séries temporais?

  - Uma **análise de séries temporais** consiste em identificar esses padrões não aleatórios.

<br>

- Quais são os padrões que podem ser encontrados na série temporal?

  - **Tendência:** são mudanças de comportamento a longo prazo, por exemplo, um crescimento da série ou mudança de hábito.

  - **Ciclos:** flutuações nos valores da variável com duração superior a um ano e que se repetem com certa periodicidade.

  - **Sazonalidade:** flutuações nos valores da variável com duração inferior a 1 ano.

  - **Variações irregulares:** flutuações inexplicáveis pelo histórico de tempo (por exemplo, catástrofes);
  

Diferentes modelos estatísticos clássicos (ARMA, ARIMA, ...) foram criados para explicar e estimar esses padrões.

<br>

Referência: 
https://medium.com/data-hackers/s%C3%A9ries-temporais-com-machine-learning-parte-1-e8fa62b82d48


Qual será nosso pipeline de análise?

- O que precisamos pensar:

  - Como podemos saber qual o melhor modelo?
  - Como podemos evitar overfitting? Ou seja, um determinado modelo se ajustar bem para uma amostra, mas não conseguir generalizar para outros casos?

### 4.1 Pipeline para treinar/avaliar modelos

#### Divisão dos conjuntos

PIPELINE PARA ANALISAR DADOS:


 **ESQUEMA**

<figure>
<center>
<img src='https://drive.google.com/uc?export=view&id=1fAtf79bgjTlCfuqJLezLmjrCC2yeofMI' />
<figcaption>ESQUEMA DE TREINO E TESTE</figcaption></center>
</figure>


Referência: https://www.kdnuggets.com/2020/05/dataset-splitting-best-practices-python.html

**Série de treinamento:** conjunto de amostras que iremos ajustar os modelos e seus parâmetros

**Série de validação:** conjunto de dados que será utilizado para avaliar o desempenho dos modelos. A partir desse conjunto, iremos identificar quais os melhores modelos e seus respectivos parâmetros

**Série de Teste:** é um conjunto de dados não enviesado, que não foi utilizado para ajustar o modelo. Depois de escolhermos o melhor modelo, precisamos avaliar se ele funciona de fato em outras amostras, na "vida real". Então é um conjunto que nos ajudará a avaliar isso

In [36]:
df.head()

,date,store,item,sales,year,month,day,dayofyear,dayofweek,weekofyear
0,2013-01-01,1,1,13,2013,1,1,1,1,1
1,2013-01-02,1,1,11,2013,1,2,2,2,1
2,2013-01-03,1,1,14,2013,1,3,3,3,1
3,2013-01-04,1,1,13,2013,1,4,4,4,1
4,2013-01-05,1,1,10,2013,1,5,5,5,1


In [ ]:
df_teste['date']

In [39]:
print(f'anos do dataframe: {df["year"].unique()}')

anos do dataframe: [2013 2014 2015 2016 2017]


In [49]:
# ano de 2017:
df.query('year ==2017')

,date,store,item,sales,year,month,day,dayofyear,dayofweek,weekofyear
1461,2017-01-01,1,1,19,2017,1,1,1,6,52
1462,2017-01-02,1,1,15,2017,1,2,2,0,1
1463,2017-01-03,1,1,10,2017,1,3,3,1,1
1464,2017-01-04,1,1,16,2017,1,4,4,2,1
1465,2017-01-05,1,1,14,2017,1,5,5,3,1
...,...,...,...,...,...,...,...,...,...,...
1821,2017-12-27,1,1,14,2017,12,27,361,2,52
1822,2017-12-28,1,1,19,2017,12,28,362,3,52
1823,2017-12-29,1,1,15,2017,12,29,363,4,52
1824,2017-12-30,1,1,27,2017,12,30,364,5,52


In [50]:
# Criando conjunto de teste:  3 últimos meses de 2017
df_test = df.query('year ==2017 & month == [10,11,12]')
df_test

,date,store,item,sales,year,month,day,dayofyear,dayofweek,weekofyear
1734,2017-10-01,1,1,21,2017,10,1,274,6,39
1735,2017-10-02,1,1,12,2017,10,2,275,0,40
1736,2017-10-03,1,1,18,2017,10,3,276,1,40
1737,2017-10-04,1,1,15,2017,10,4,277,2,40
1738,2017-10-05,1,1,20,2017,10,5,278,3,40
...,...,...,...,...,...,...,...,...,...,...
1821,2017-12-27,1,1,14,2017,12,27,361,2,52
1822,2017-12-28,1,1,19,2017,12,28,362,3,52
1823,2017-12-29,1,1,15,2017,12,29,363,4,52
1824,2017-12-30,1,1,27,2017,12,30,364,5,52


In [51]:
# Criando conjunto de validação:  meses 7, 8 e 9 de 2017
df_val = df.query('year ==2017 & month == [7,8,9]')
df_val

,date,store,item,sales,year,month,day,dayofyear,dayofweek,weekofyear
1642,2017-07-01,1,1,31,2017,7,1,182,5,26
1643,2017-07-02,1,1,38,2017,7,2,183,6,26
1644,2017-07-03,1,1,18,2017,7,3,184,0,27
1645,2017-07-04,1,1,23,2017,7,4,185,1,27
1646,2017-07-05,1,1,23,2017,7,5,186,2,27
...,...,...,...,...,...,...,...,...,...,...
1729,2017-09-26,1,1,18,2017,9,26,269,1,39
1730,2017-09-27,1,1,27,2017,9,27,270,2,39
1731,2017-09-28,1,1,23,2017,9,28,271,3,39
1732,2017-09-29,1,1,28,2017,9,29,272,4,39


In [ ]:
# Criando conjunto de validação:
df_train = df.query('date < "2017-07-01"')
df_train

In [61]:
from plotly.subplots import make_subplots

graf0 = make_subplots(rows=1, cols=1)

graf0.add_trace(go.Scatter(x = df_train['date'],y = df_train['sales'].values,
mode = 'lines',line = dict(width=2),name = 'treino'), row=1,col=1)  # grÃ¡fico da variÃ¡vel    

graf0.add_trace(go.Scatter(x = df_val['date'],y = df_val['sales'].values,
mode = 'lines',line = dict(width=2),name = 'validaçãoo'), row=1,col=1)  # grÃ¡fico da variÃ¡vel    

graf0.add_trace(go.Scatter(x = df_test['date'],y = df_test['sales'].values,
mode = 'lines',line = dict(width=2),name = 'teste'), row=1,col=1)  # grÃ¡fico da variÃ¡vel    



graf0.update_layout(height=400, width=1100,title={'text': 'Séries'},
        plot_bgcolor = 'white',
        font = {'family': 'Arial','size': 16,'color': '#929497'})  # fonte dos textos: família, o tamanho, a cor

graf0.update_xaxes( showgrid = False, gridwidth=1, gridcolor='lightgray',
showline=True, linewidth=1, linecolor='#929497'
)

graf0.update_yaxes(showgrid=False, gridwidth=1, gridcolor='lightgray',
showline=True, linewidth=1, linecolor='#929497')

#### Métrica de avaliação

Como iremos saber se um modelo é melhor do que o outro?

- Precisamos de uma métrica (medida de erro) que irá nos mostrar o quanto estamos acertando ou errando com nosso modelo. As métricas são basicamente medidas que envolvem calcular a diferença entre o que o modelo estima e o valor real do modelo.

- Para nosso estudo, iremos utilizar a métrica RMSE, dada pela fórmula:

<br>

 **RMSE = $\sqrt{ \frac{1}{N}\sum_{i=1}^{N} (predict_{i} - actual_{i})^2}$**


In [ ]:
import sklearn.metrics
#rmse = sklearn.metrics.mean_squared_error(y_true, y_pred, sample_weight=None, multioutput='uniform_average', squared=False)

#### Primeiros modelos e análises estatísticas de séries temporais

In [ ]:
## Repetir processo até encontrar melhor modelo:

  # 1- ajustar um modelo na série de treino
  # 2 - prever resultados da série de teste
  # 3 - avaliar seu desempenho + escolher melhores parâmetros na série de validação com a métrica de erro

## Escolhido modelo + melhores parâmetros, prever no conjunto de teste e avaliar desempenho (generalização)


In [54]:
df_train.head(3)

,date,store,item,sales,year,month,day,dayofyear,dayofweek,weekofyear
0,2013-01-01,1,1,13,2013,1,1,1,1,1
1,2013-01-02,1,1,11,2013,1,2,2,2,1
2,2013-01-03,1,1,14,2013,1,3,3,3,1


##### Modelo de Regressão Linear

In [66]:
df_val.index

Int64Index([1642, 1643, 1644, 1645, 1646, 1647, 1648, 1649, 1650, 1651, 1652,
            1653, 1654, 1655, 1656, 1657, 1658, 1659, 1660, 1661, 1662, 1663,
            1664, 1665, 1666, 1667, 1668, 1669, 1670, 1671, 1672, 1673, 1674,
            1675, 1676, 1677, 1678, 1679, 1680, 1681, 1682, 1683, 1684, 1685,
            1686, 1687, 1688, 1689, 1690, 1691, 1692, 1693, 1694, 1695, 1696,
            1697, 1698, 1699, 1700, 1701, 1702, 1703, 1704, 1705, 1706, 1707,
            1708, 1709, 1710, 1711, 1712, 1713, 1714, 1715, 1716, 1717, 1718,
            1719, 1720, 1721, 1722, 1723, 1724, 1725, 1726, 1727, 1728, 1729,
            1730, 1731, 1732, 1733],
           dtype='int64')

In [70]:
#Separando X_train, y_train, X_val e y_val
x_t = np.array(df_train.index).reshape(-1, 1)    # X_treino são os index (0...1641)
y_t = df_train['sales'] # y_treino são as vendas (começando do index até 0...1641)
x_v = np.array(df_val.index).reshape(-1, 1)      # X_validacao são os index (1642...1733)
y_v = df_val['sales']   # y_validacao são as vendas (começando do index até1642...1733)

In [73]:
# importar a biblioteca de regressão linear
from sklearn.linear_model import LinearRegression    

# chamando o módulo da regressão
reg = LinearRegression()                              

# ajustando os dados na série de treino
reg.fit(x_t,y_t)                             

# prevendo na série de teste
y_pred = reg.predict(x_v)


In [76]:
# Avaliar nossas métricas obtidas no modelo
import sklearn.metrics
rmse_modelo1 = sklearn.metrics.mean_squared_error(y_true = y_v, y_pred = y_pred, sample_weight=None, multioutput='uniform_average', squared=False)
rmse_modelo1

6.843621482555741

In [85]:
from plotly.subplots import make_subplots

graf0 = make_subplots(rows=1, cols=1)

graf0.add_trace(go.Scatter(x = df_train['date'],y = df_train['sales'].values,
mode = 'lines',line = dict(width=2),name = 'treino'), row=1,col=1)  # grÃ¡fico da variÃ¡vel    

graf0.add_trace(go.Scatter(x = df_val['date'],y = df_val['sales'].values,
mode = 'lines',line = dict(width=2),name = 'validaçãoo'), row=1,col=1)  # grÃ¡fico da variÃ¡vel    

graf0.add_trace(go.Scatter(x = df_test['date'],y = df_test['sales'].values,
mode = 'lines',line = dict(width=2),name = 'teste'), row=1,col=1)  # grÃ¡fico da variÃ¡vel    

# ajuste do treino
graf0.add_trace(go.Scatter(x = df_train['date'],y = reg.predict(x_t),
mode = 'lines',line = dict(color = 'red', width=3),name = 'ajuste treino'), row=1,col=1)  # grÃ¡fico da variÃ¡vel  

# ajuste da validação
graf0.add_trace(go.Scatter(x = df_val['date'],y = reg.predict(x_v),
mode = 'lines',line = dict(color = 'black', width=3),name = 'ajuste validação'), row=1,col=1)  # grÃ¡fico da variÃ¡vel  


graf0.update_layout(height=400, width=1100,title={'text': 'Modelo de Regressão Linear'},
        plot_bgcolor = 'white',
        font = {'family': 'Arial','size': 16,'color': '#929497'})  # fonte dos textos: família, o tamanho, a cor

graf0.update_xaxes( showgrid = False, gridwidth=1, gridcolor='lightgray',
showline=True, linewidth=1, linecolor='#929497'
)

graf0.update_yaxes(showgrid=False, gridwidth=1, gridcolor='lightgray',
showline=True, linewidth=1, linecolor='#929497')

Próximos passos:

- Modelos de médias móveis
- ARIMA, SARIMA,...
- Geração de desvios, avaliação de distribuições e MC
- Feature transforming + Modelos de ML